In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import random
from itertools import product
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


sys.path.extend(['../src',
                '../data',])

from utils import run_nested_cv, run_approach, process_fold

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("CPU")
    
seed_num = 42
random.seed(seed_num)
np.random.seed(seed_num)
tf.random.set_seed(seed_num)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

#data
left = pd.read_csv('../data/left.csv')
right = pd.read_csv('../data/right.csv')
affected = pd.read_csv('../data/affected.csv')
unaffected = pd.read_csv('../data/unaffected.csv')

#limb
right['limb'] = 1
left['limb'] = 2
affected['limb'] = 3
unaffected['limb'] = 4

affected['subject'] = affected['subject'] + 10
unaffected['subject'] = unaffected['subject'] + 10

combined = pd.concat([unaffected, 
                      affected, 
                      right, 
                      left], 
                      ignore_index=True)

architecture = {
               'num_filters': [50, 25],
               'filter_sizes': [5, 5],
               'pool_sizes': [5, 5]
               }

In [ ]:
approach = 1
filename = f'../results/approach1/limb_nested_cv_left.csv'

window_size = 100
epochs = 200
batch_size = 32

#For approach1, supervised (class_loss = True), unsupervised (class_loss = False)
class_loss = True
plot_hist = False

X = left[['ax', 'ay', 'az']]
y = left['gnd']
subjects = left['subject']
limbs = left['limb']

hp_grid = {
    'activation': ['relu', 'tanh'],
    'kernel_init': [tf.keras.initializers.HeNormal(seed=42),
                    tf.keras.initializers.GlorotNormal(seed=42)],
    'bias_init': [tf.keras.initializers.Zeros()],
    'optimizer': ['adam', 'nadam'],
    'loss': [tf.keras.losses.mean_squared_error,
             tf.keras.losses.mean_absolute_error],
    'learning_rate': [0.001, 0.0001],
    'batch_norm': [True, False]
} 

results_df = run_nested_cv(window_size=window_size,
                           X=X,
                           y=y,
                           subjects=subjects,
                           limbs=limbs,
                           architecture=architecture,
                           hp_grid=hp_grid,
                           epochs=epochs,
                           batch_size=batch_size,
                           class_loss=class_loss,
                           plot_hist=plot_hist,
                           approach=approach)

summary_results = results_df.groupby(['limb'] + list(hp_grid.keys()))['youden_index'].mean().reset_index()
summary_results.to_csv(filename, index=False)
print(np.mean(results_df['youden_index']), np.std(results_df['youden_index']))